In [ ]:
import numpy as np
import pandas as pd 

from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier

from sentence_transformers import SentenceTransformer
import torch
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
tqdm.pandas()

In [2]:
path="../data/"
train = pd.read_parquet(path+"train.parquet")

train, test = train_test_split(train, test_size=0.2, random_state=1)
eval, test = train_test_split(test, test_size=0.5, random_state=1)

sub = pd.read_csv(path+"sample_submission.csv")

In [3]:
train.head()

,id,prompt,response_a,response_b,winner,model_a,model_b,language
18632,623e2aa50f13aece7fb1c60cf555d8c7d290692f679655...,Phân tích kết quả sản xuất (KQSX) về chất lượn...,Để phân tích kết quả sản xuất (KQSX) về chất l...,Phân tích kết quả sản xuất (KQSX) về chất lượn...,model_a,yi-lightning,internlm2_5-20b-chat,Vietnamese
34859,b87079eef09873032f653314cfddd05e7bf77e4e6d0449...,Write a sample of Old Uyghur,Providing a truly representative sample of Old...,"I apologize, but I cannot generate an authenti...",model_a,gemini-1.5-flash-002,claude-3-5-sonnet-20240620,English
43764,e721d5bfbff8e05e2ffa92e0137a570c1b1b7fa3f7bbf7...,Write a poem in the style of Ada Limon.,**Under the Bruise of the World**\n\nIn the ga...,Here's a poem inspired by the style of Ada Lim...,model_b,reka-core-20240722,grok-2-2024-08-13,English
22541,77350a005d0195fa951b9fb073b018f2f6424a58ee58b5...,"describe the way ""golden slumbers"" by the beat...","Describing the way ""Golden Slumbers"" by The Be...","For a deaf person who has never heard ""Golden ...",model_a,jamba-1.5-large,deepseek-coder-v2-0724,English
37157,c43d63ee99a8b776065f410e4a1e0791a4449230cfd648...,かけっこをしています。あなたは3位の人を抜かし ました。今あなたは何位？,3位です。あなたは3位の人を抜かしたので、今あなたは3位になります。,はい、かけっこの状況について考えてみましょう。\n\n問題文から、以下のことがわかります。\...,model_a,o1-preview,claude-3-opus-20240229,Japanese


In [4]:
test.head()

,id,prompt,response_a,response_b,winner,model_a,model_b,language
7419,271ba404fc25609b8ceb6f37c3375f278c42e2479514ea...,Улучшь сообщение: Добрый вечер! \nЭто Алексей ...,"Добрый вечер!\n\nМеня зовут Алексей Королев, я...","Добрый вечер!\n\nЯ, Алексей Королев, заместите...",model_b,qwen-max-0919,c4ai-aya-expanse-32b,Russian
39112,ce7be103ac8f040accc9b8ec421a6cbf78189c503671df...,i have a notification icon in my webpage. when...,Certainly! To implement a notification system ...,Here's a simple example of how you can achieve...,model_b,qwen2.5-72b-instruct,llama-3.1-405b-instruct-bf16,English
33946,b39725e0a7992eb0c5f34542f960727a4fd306741568fb...,I have a raspberry pi that is booted from USB....,**Hotplugging the SD Card on a Raspberry Pi**\...,"No, you generally **cannot** safely hotplug th...",model_a,llama-3.1-8b-instruct,gemini-1.5-pro-exp-0827,English
35392,bb33f84e8e85da9e41fa271f6210fdffba0400d725a04c...,"Imagine video on YouTube ""Asking people what i...",Here are some imagined YouTube comments relati...,Here are some comments that might appear under...,model_a,gemini-1.5-flash-002,gemma-2-27b-it,English
10254,364c8146a6a236fbdfa34492dba10405a0e448d3062316...,generate a wine description,Please provide some context! To generate a co...,**Wine Description:**\n\n**Name:** Sunset on t...,model_b,gemini-1.5-flash-8b-001,llama-3.1-70b-instruct,English


In [10]:
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Dense({'in_features': 768, 'out_features': 512, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
)

In [16]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [6]:
class QADataset(Dataset):
    def __init__(self, data, device):
        self.data = data
        self.device = device
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        return (
            row["prompt"],
            row["response_a"],
            row["response_b"],
        )
    

In [20]:
device = 'cuda'

# model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

model = model.to(device)

batch_size = 128
train_data_loader = DataLoader(QADataset(train, device), batch_size=batch_size, shuffle=True, drop_last=True)
eval_data_loader = DataLoader(QADataset(eval, device), batch_size=batch_size, drop_last=True)
test_data_loader = DataLoader(QADataset(test, device), batch_size=batch_size, drop_last=True)



In [18]:
@torch.no_grad()
def get_dataloader_embeddings(dataloader: DataLoader, model):
    embeddings = {
        "prompt": [], 
        "response_a": [], 
        "response_b": [], 
    }

    model.eval()

    for batch in tqdm(dataloader):
        prompts = batch[0]
        responses_a = batch[1]
        responses_b = batch[2]

        prompts_embs = model.encode(prompts, convert_to_tensor=True)
        response_a_embs = model.encode(responses_a, convert_to_tensor=True)
        response_b_embs = model.encode(responses_b, convert_to_tensor=True)

        embeddings["prompt"].append(prompts_embs.detach().cpu().numpy())
        embeddings["response_a"].append(response_a_embs.detach().cpu().numpy())
        embeddings["response_b"].append(response_b_embs.detach().cpu().numpy())
    
    return embeddings

In [19]:
train_embeddings = get_dataloader_embeddings(train_data_loader, model)
eval_embeddings = get_dataloader_embeddings(eval_data_loader, model)
test_embeddings = get_dataloader_embeddings(test_data_loader, model)

100%|██████████| 37/37 [00:06<00:00,  5.93it/s]


In [21]:
train_embeddings['prompt'][0].shape

(128, 512)

In [21]:
def aggregate_dataloader_embeddings(embeddings: dict[str, list[np.ndarray]]):
    prompts_embs = np.concatenate(embeddings['prompt'])
    response_a_embs = np.concatenate(embeddings['response_a'])
    response_b_embs = np.concatenate(embeddings['response_b'])

    all_embs = np.concatenate([prompts_embs, response_a_embs, response_b_embs], axis=1)
    return all_embs

train_features = aggregate_dataloader_embeddings(train_embeddings)
eval_features = aggregate_dataloader_embeddings(eval_embeddings)
test_features = aggregate_dataloader_embeddings(test_embeddings)

In [22]:
X_train = train_features
y_train = train['winner'].iloc[:len(X_train)]

X_eval = eval_features
y_eval = eval['winner'].iloc[:len(X_eval)]

X_test = test_features
y_test = test['winner'].iloc[:len(X_test)]


print(len(X_train), len(y_train))
print(len(X_eval), len(y_eval))
print(len(X_test), len(y_test))

38656 38656
4736 4736
4736 4736


In [24]:
cbm = CatBoostClassifier(depth=6, random_state=42, eval_metric="Accuracy", verbose=100)
cbm.fit(X_train, y_train, eval_set=(X_eval, y_eval))

Learning rate set to 0.078154
0:	learn: 0.5194795	test: 0.4985220	best: 0.4985220 (0)	total: 27.3ms	remaining: 27.3s
100:	learn: 0.6710213	test: 0.4890203	best: 0.5023226 (7)	total: 2.64s	remaining: 23.5s
200:	learn: 0.7543460	test: 0.4970439	best: 0.5023226 (7)	total: 5.19s	remaining: 20.6s
300:	learn: 0.8181395	test: 0.4959882	best: 0.5023226 (7)	total: 7.99s	remaining: 18.6s
400:	learn: 0.8601252	test: 0.5027449	best: 0.5050676 (378)	total: 10.8s	remaining: 16.2s
500:	learn: 0.8926169	test: 0.4997889	best: 0.5073902 (441)	total: 13.9s	remaining: 13.8s
600:	learn: 0.9182792	test: 0.4902872	best: 0.5073902 (441)	total: 17s	remaining: 11.3s
700:	learn: 0.9353270	test: 0.4955659	best: 0.5073902 (441)	total: 19.8s	remaining: 8.45s
800:	learn: 0.9500207	test: 0.4951436	best: 0.5073902 (441)	total: 22.7s	remaining: 5.63s
900:	learn: 0.9603942	test: 0.4978885	best: 0.5073902 (441)	total: 25.5s	remaining: 2.8s
999:	learn: 0.9682844	test: 0.5002111	best: 0.5073902 (441)	total: 28.4s	remaining